#### Here first we will pass context and question at same time

In [24]:
from langchain_community.vectorstores.faiss import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_community.embeddings.huggingface import HuggingFaceBgeEmbeddings
from langchain_community.llms.ollama import Ollama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

In [25]:
db = FAISS.from_texts(
    ["Anurag Worked at JellyFish Technologies"],
    HuggingFaceBgeEmbeddings()
)

retriever = db.as_retriever()

In [26]:
llm = Ollama(model='llama2')

In [27]:
prompt = ChatPromptTemplate.from_template(
""" 
Answer the question based only on the following context:

context: {context}


Question: {question}
"""
)

retrieval_chain = (
    {"context":retriever, "question":RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [28]:
retrieval_chain.invoke("Where does Anurag Works ?")

'based on the provided context, the answer to the question "Where does Anurag work?" is:\nJellyFish Technologies'

#### Choosing Particular key while passing final dictionary during invokation

In [29]:
from operator import itemgetter
from langchain_community.vectorstores.faiss import FAISS
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


In [31]:
db = FAISS.from_texts(
    ['Anurag works at JellyFish Technologies.','It is located in Noida'],
    HuggingFaceBgeEmbeddings()
)
retriever = db.as_retriever()

In [33]:
prompt = ChatPromptTemplate.from_template(
    """ 
Answer the question based on the following context only:

context: {context}

Question: {question}

Answer in the following language: {language}
"""
)


llm = Ollama(model="llama2")

In [35]:
retrieval_chain = (
    {"context":itemgetter('question') | retriever,
     "question":itemgetter("question"),
    "language":itemgetter('language')
    }
    | prompt
    | llm
    | StrOutputParser()
)




In [36]:
retrieval_chain.invoke({"question":"where does Anurag Works","language":"English"})

' Based on the provided context, Anurag works at JellyFish Technologies in Noida.'

### Passing two chains at a same time using Runnable Parallel

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_community.llms.ollama import Ollama

In [3]:
model = Ollama(model='llama2',temperature=0.9)

joke_chain =  ChatPromptTemplate.from_template(" tell me a joke about {topic}") | model

poem_chain = ChatPromptTemplate.from_template("Write a 2- line poem about {topic}") | model

map_chain = RunnableParallel(joke = joke_chain, king = poem_chain)

map_chain.invoke({"topic": "bad days"})

{'joke': "Sure, here's one:\n\nWhy did the raincloud go to the therapist?\n\nBecause it was having a really cloudy day!",
 'king': '\nRainy days drag on and on,\nStealing smiles and laughter away.'}